In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

# --------------------------------------------------
# PASO 1: Cargar el DataFrame y convertir 'date_time' a datetime
# --------------------------------------------------
df = pd.read_csv("../../data/processed/navegacion_clientes_experimento_limpio.csv")

# Obtener estadísticas descriptivas de todas las columnas
desc = df.describe(include='all').T

# Añadir información sobre valores nulos y únicos
desc['missing'] = df.isnull().sum()
desc['unique'] = df.nunique()

# Formatear la tabla en formato Markdown
markdown_table = desc.to_markdown()

# Mostrar la tabla en formato Markdown
print(markdown_table)

|                  |   count |   unique | top                         |   freq |            mean |             std |     min |             25% |             50% |             75% |              max |   missing |
|:-----------------|--------:|---------:|:----------------------------|-------:|----------------:|----------------:|--------:|----------------:|----------------:|----------------:|-----------------:|----------:|
| client_id        |  216498 |    35203 | nan                         |    nan |     5.02597e+06 |     2.87813e+06 |   555   |     2.52982e+06 |     5.06503e+06 |     7.51059e+06 |      9.99963e+06 |         0 |
| visitor_id       |  216498 |    38625 | 181494464_67394447335       |     58 |   nan           |   nan           |   nan   |   nan           |   nan           |   nan           |    nan           |         0 |
| visit_id         |  216498 |    46603 | 428529357_6959155752_124163 |     42 |   nan           |   nan           |   nan   |   nan           |   nan  

In [6]:
# Obtener las primeras filas del DataFrame
head_df = df.head(20)

# Añadir el tipo de cada columna al encabezado
column_types = df.dtypes
head_df.columns = [f"{col} ({dtype})" for col, dtype in zip(head_df.columns, column_types)]

# Formatear la tabla en formato Markdown
markdown_table = head_df.to_markdown(index=False)

# Mostrar la tabla en formato Markdown
print(markdown_table)

|   client_id (int64) |   visitor_id (object) |            visit_id (object) | process_step (object)   | date_time (object)   |   clnt_tenure_yr (int64) |   clnt_tenure_mnth (int64) |   clnt_age (float64) | gendr (object)   |   num_accts (int64) |   bal (float64) |   calls_6_mnth (int64) |   logons_6_mnth (int64) | variation (object)   |
|--------------------:|----------------------:|-----------------------------:|:------------------------|:---------------------|-------------------------:|---------------------------:|---------------------:|:-----------------|--------------------:|----------------:|-----------------------:|------------------------:|:---------------------|
|             9988021 |  580560515_7732621733 | 781255054_21935453173_531117 | step_3                  | 2017-04-17 15:27:07  |                        5 |                         64 |                 79   | U                |                   2 |        189024   |                      1 |                       4 | Tes

In [7]:
df["date_time"] = pd.to_datetime(df["date_time"])

# Filtrar registros relevantes: 'start' y 'confirm'
df_times = df[df["process_step"].isin(["start", "confirm"])].copy()

# Agrupar a nivel de sesión (visit_id y variation)
def get_start(x):
    return x[df_times.loc[x.index, "process_step"] == "start"].min()

def get_confirm(x):
    return x[df_times.loc[x.index, "process_step"] == "confirm"].max()

df_session_times = (
    df_times.groupby(["visit_id", "variation"], as_index=False)
            .agg(start_time=("date_time", get_start),
                 confirm_time=("date_time", get_confirm))
)

# Eliminar sesiones sin confirm (donde confirm_time es NaT)
df_session_times = df_session_times.dropna(subset=["confirm_time"])

# Calcular Time to Complete (TTC)
df_session_times["TTC_minutes"] = (df_session_times["confirm_time"] - df_session_times["start_time"]).dt.total_seconds() / 60

# Separar los grupos Test y Control
ttc_test = df_session_times[df_session_times["variation"] == "Test"]["TTC_minutes"]
ttc_control = df_session_times[df_session_times["variation"] == "Control"]["TTC_minutes"]

# Verificar si ttc_test y ttc_control contienen valores NaN o están vacíos
print("Número de valores NaN en TTC Test:", ttc_test.isna().sum())
print("Número de valores NaN en TTC Control:", ttc_control.isna().sum())
print("Número de valores en TTC Test:", len(ttc_test))
print("Número de valores en TTC Control:", len(ttc_control))

# Eliminar valores NaN de ttc_test y ttc_control
ttc_test = ttc_test.dropna()
ttc_control = ttc_control.dropna()

# Asegurarse de que ttc_test y ttc_control no están vacíos
if len(ttc_test) == 0 or len(ttc_control) == 0:
    raise ValueError("Uno de los grupos (Test o Control) está vacío después de eliminar valores NaN.")

# Realizar el test t de muestras independientes
t_stat, p_value_two_tailed = stats.ttest_ind(ttc_test, ttc_control, equal_var=False)

# Convertir a p-value one-tailed (para H1: Test < Control)
if t_stat < 0:
    p_value_one_tailed = p_value_two_tailed / 2
else:
    p_value_one_tailed = 1 - p_value_two_tailed / 2

print("=== Time to Complete (TTC) por sesión (en minutos) ===")
print(f"Media TTC Test   : {ttc_test.mean():.2f} minutos")
print(f"Media TTC Control: {ttc_control.mean():.2f} minutos")
print(f"t-statistic      : {t_stat:.4f}")
print(f"p-value (one-tailed): {p_value_one_tailed:.6f}")

alpha = 0.05
if p_value_one_tailed < alpha:
    print(f"\nConclusión: Rechazamos H0. El grupo Test tiene un TTC significativamente menor que Control (p < {alpha}).")
else:
    print(f"\nConclusión: No rechazamos H0. No se evidencia que el grupo Test tenga un TTC menor que Control (p >= {alpha}).")

Número de valores NaN en TTC Test: 1990
Número de valores NaN en TTC Control: 661
Número de valores en TTC Test: 14365
Número de valores en TTC Control: 10782
=== Time to Complete (TTC) por sesión (en minutos) ===
Media TTC Test   : 6.95 minutos
Media TTC Control: 6.72 minutos
t-statistic      : 1.9864
p-value (one-tailed): 0.976497

Conclusión: No rechazamos H0. No se evidencia que el grupo Test tenga un TTC menor que Control (p >= 0.05).


## Conclusiones

Los resultados son coherentes con lo que se observa en los datos. En tu análisis, la media del TTC para el grupo Test (6.93 minutos) es ligeramente mayor que la del grupo Control (6.73 minutos). Esto implica que, en promedio, los usuarios del grupo Test tardan un poco más en completar el proceso, lo que es contrario a la hipótesis alternativa que esperábamos (Test < Control).

Por ello, el t-statistic es positivo (2.0926) y el p-value one-tailed es muy alto (0.981805), lo que nos lleva a no rechazar H0. Es decir, no hay evidencia de que el grupo Test tenga un TTC menor que el grupo Control.

En resumen, tu modificación es correcta y los resultados indican que, según los datos actuales, la nueva interfaz (Test) no mejora (reduciendo) el tiempo de finalización en comparación con la versión tradicional (Control).

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# --------------------------------------------------
# PASO 1: Cargar el DataFrame y convertir 'date_time' a datetime
# --------------------------------------------------
df = pd.read_csv("../../data/processed/navegacion_clientes_experimento_limpio.csv")
df["date_time"] = pd.to_datetime(df["date_time"])

# --------------------------------------------------
# PASO 2: Filtrar registros relevantes: 'start' y 'confirm'
# --------------------------------------------------
df_times = df[df["process_step"].isin(["start", "confirm"])].copy()

# --------------------------------------------------
# PASO 3: Agrupar a nivel de sesión (visit_id y variation) usando groupby.apply
# --------------------------------------------------
def session_times(group):
    # Se obtiene el primer "start" y el último "confirm" dentro de cada grupo
    start_time = group.loc[group["process_step"] == "start", "date_time"].min()
    confirm_time = group.loc[group["process_step"] == "confirm", "date_time"].max()
    return pd.Series({"start_time": start_time, "confirm_time": confirm_time})

df_session_times = (
    df_times.groupby(["visit_id", "variation"], as_index=False, observed=False)
            .apply(session_times, include_groups=False)
).reset_index(drop=True)

# Eliminar sesiones sin confirm_time (donde confirm_time es NaT)
df_session_times = df_session_times.dropna(subset=["confirm_time"])

# --------------------------------------------------
# PASO 4: Calcular Time to Complete (TTC) en minutos para cada sesión
# --------------------------------------------------
df_session_times["TTC_minutes"] = (
    (df_session_times["confirm_time"] - df_session_times["start_time"]).dt.total_seconds() / 60
)

# --------------------------------------------------
# PASO 5: Separar los grupos Test y Control
# --------------------------------------------------
ttc_test = df_session_times[df_session_times["variation"] == "Test"]["TTC_minutes"].dropna()
ttc_control = df_session_times[df_session_times["variation"] == "Control"]["TTC_minutes"].dropna()

# Calcular medias
mean_test = ttc_test.mean()
mean_control = ttc_control.mean()

print("=== Time to Complete (TTC) por sesión (en minutos) ===")
print("Media TTC Test   : {:.2f} minutos".format(mean_test))
print("Media TTC Control: {:.2f} minutos".format(mean_control))

# --------------------------------------------------
# PASO 6: Plantear Hipótesis y realizar el test
# --------------------------------------------------
# Hipótesis:
#   H0: mu_Test = mu_Control
#   H1: mu_Test < mu_Control   (menor TTC en Test implica mejor UX)
t_stat, p_value_two_tailed = stats.ttest_ind(ttc_test, ttc_control, equal_var=False)

# Ajustar p-value para test one-tailed (H1: Test < Control)
if t_stat < 0:
    p_value_one_tailed = p_value_two_tailed / 2
else:
    p_value_one_tailed = 1 - p_value_two_tailed / 2

print("t-statistic      : {:.4f}".format(t_stat))
print("p-value (one-tailed): {:.6f}".format(p_value_one_tailed))

alpha = 0.05
if p_value_one_tailed < alpha:
    print("\nConclusión: Rechazamos H0. El grupo Test tiene un TTC significativamente menor que Control (p < {:.2f}).".format(alpha))
else:
    print("\nConclusión: No rechazamos H0. No se evidencia que el grupo Test tenga un TTC menor que Control (p >= {:.2f}).".format(alpha))


=== Time to Complete (TTC) por sesión (en minutos) ===
Media TTC Test   : 6.95 minutos
Media TTC Control: 6.72 minutos
t-statistic      : 1.9864
p-value (one-tailed): 0.976497

Conclusión: No rechazamos H0. No se evidencia que el grupo Test tenga un TTC menor que Control (p >= 0.05).


## Interpretación:

- La media del Time to Complete (TTC) en el grupo Test (6.95 minutos) es ligeramente mayor que en el grupo Control (6.72 minutos), lo que no respalda la hipótesis de que la nueva interfaz (Test) reduce el tiempo de finalización.
- El t-statistic de 1.9864 indica que la diferencia entre las medias no es suficientemente grande para ser estadísticamente significativa.
- El p-value one-tailed de 0.976497 es muy alto (mucho mayor que 0.05), lo que significa que la probabilidad de observar esta diferencia si realmente Test fuera más rápido es muy baja.
- Conclusión: No se rechaza H0 → No hay evidencia suficiente para afirmar que el grupo Test tenga un TTC menor que el grupo Control.

## ¿Es un resultado válido?

Sí, es completamente válido. En experimentos A/B testing, es común que algunas métricas mejoren significativamente (rechazo de H0) mientras que otras no muestran cambios estadísticamente relevantes. En este caso, el Completion Rate (CR) y Cost-Effectiveness (CE) podrían haber mostrado mejoras significativas, mientras que TTC no. Esto sugiere que la nueva interfaz puede haber aumentado la tasa de finalización, pero sin necesariamente acelerar el proceso.

Si deseas analizar por qué Test no es más rápido, podrías hacer segmentaciones por edad, experiencia del usuario (antigüedad en la plataforma), número de cuentas, o incluso explorar si hay más clics o pasos adicionales en Test que puedan estar ralentizando la finalización.